In [6]:
import imutils
import time
import cv2
import csv
import os

cascade = 'haarcascade_frontalface_default.xml'
detector = cv2.CascadeClassifier(cascade)

Name = str(input("Enter your Name : "))
Roll_Number = int(input("Enter your Roll_Number : "))
dataset = 'dataset'
sub_data = Name
path = os.path.join(dataset, sub_data)

if not os.path.isdir(path):
    os.mkdir(path)
    print(sub_data)

info = [str(Name), str(Roll_Number)]
with open('student.csv', 'a') as csvFile:#writing into existing csv
    write = csv.writer(csvFile)
    write.writerow(info)#writing into csv
csvFile.close()

print("Starting video stream...")
cam = cv2.VideoCapture(0)
time.sleep(2.0)
total = 0

while total < 50:#store 50 images 
    print(total)
    _, frame = cam.read()
    img = imutils.resize(frame, width=400)
    rects = detector.detectMultiScale(
        cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), scaleFactor=1.1,
        minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in rects:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        p = os.path.sep.join([path, "{}.png".format(
            str(total).zfill(5))])
        cv2.imwrite(p, img)
        total += 1

    cv2.imshow("Frame", frame)#for clos
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cam.release()
cv2.destroyAllWindows()


Enter your Name : shivani2
Enter your Roll_Number : 34
shivani2
Starting video stream...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
35
36
37
38
39
40
40
41
42
43
44
45
46
47
48
49


In [8]:
from imutils import paths
import numpy as np
import imutils
import pickle
import cv2
import os

dataset = "Dataset"

embeddingFile = "output/embeddings.pickle" #initial name for embedding file
embeddingModel = "openface_nn4.small2.v1.t7" #initializing model for embedding Pytorch

#initialization of caffe model for face detection
prototxt = "model/deploy.prototxt"
model =  "model/res10_300x300_ssd_iter_140000.caffemodel"

#loading caffe model for face detection
#detecting face from Image via Caffe deep learning
detector = cv2.dnn.readNetFromCaffe(prototxt, model)

#loading pytorch model file for extract facial embeddings
#extracting facial embeddings via deep learning feature extraction
embedder = cv2.dnn.readNetFromTorch(embeddingModel)

#gettiing image paths
imagePaths = list(paths.list_images(dataset))

#initialization
knownEmbeddings = []
knownNames = []
total = 0
conf = 0.5

#we start to read images one by one to apply face detection and embedding
for (i, imagePath) in enumerate(imagePaths):
	print("Processing image {}/{}".format(i + 1,len(imagePaths)))
	name = imagePath.split(os.path.sep)[-2]
	image = cv2.imread(imagePath)
	image = imutils.resize(image, width=600)
	(h, w) = image.shape[:2]
	#converting image to blob for dnn face detection
	imageBlob = cv2.dnn.blobFromImage(
		cv2.resize(image, (300, 300)), 1.0, (300, 300),(104.0, 177.0, 123.0), swapRB=False, crop=False)

	#setting input blob image
	detector.setInput(imageBlob)
	#prediction the face
	detections = detector.forward()

	if len(detections) > 0:
		i = np.argmax(detections[0, 0, :, 2])
		confidence = detections[0, 0, i, 2]

		if confidence > conf:
			#ROI range of interest
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")
			face = image[startY:endY, startX:endX]
			(fH, fW) = face.shape[:2]

			if fW < 20 or fH < 20:
				continue
			#image to blob for face
			faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
			#facial features embedder input image face blob
			embedder.setInput(faceBlob)
			vec = embedder.forward()
			knownNames.append(name)
			knownEmbeddings.append(vec.flatten())
			total += 1

print("Embedding:{0} ".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open(embeddingFile, "wb")
f.write(pickle.dumps(data))
f.close()
print("Process Completed")

Processing image 1/100
Processing image 2/100
Processing image 3/100
Processing image 4/100
Processing image 5/100
Processing image 6/100
Processing image 7/100
Processing image 8/100
Processing image 9/100
Processing image 10/100
Processing image 11/100
Processing image 12/100
Processing image 13/100
Processing image 14/100
Processing image 15/100
Processing image 16/100
Processing image 17/100
Processing image 18/100
Processing image 19/100
Processing image 20/100
Processing image 21/100
Processing image 22/100
Processing image 23/100
Processing image 24/100
Processing image 25/100
Processing image 26/100
Processing image 27/100
Processing image 28/100
Processing image 29/100
Processing image 30/100
Processing image 31/100
Processing image 32/100
Processing image 33/100
Processing image 34/100
Processing image 35/100
Processing image 36/100
Processing image 37/100
Processing image 38/100
Processing image 39/100
Processing image 40/100
Processing image 41/100
Processing image 42/100
P

In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import pickle

#initilizing of embedding & recognizer
embeddingFile = "output/embeddings.pickle"
#New & Empty at initial
recognizerFile = "output/recognizer.pickle"
labelEncFile = "output/le.pickle"

print("Loading face embeddings...")
data = pickle.loads(open(embeddingFile, "rb").read())

print("Encoding labels...")
labelEnc = LabelEncoder()
labels = labelEnc.fit_transform(data["names"])


print("Training model...")
recognizer = SVC(C=1.0, kernel="linear", probability=True)
recognizer.fit(data["embeddings"], labels)
print("trsining done")
f = open(recognizerFile, "wb")
f.write(pickle.dumps(recognizer))
f.close()

f = open(labelEncFile, "wb")
f.write(pickle.dumps(labelEnc))
f.close()
print("Done")

Loading face embeddings...
Encoding labels...
Training model...
trsining done
Done


In [12]:
import numpy as np
import imutils
import pickle
import time
import cv2

embeddingModel = "openface_nn4.small2.v1.t7"

embeddingFile = "output/embeddings.pickle"
recognizerFile = "output/recognizer.pickle"
labelEncFile = "output/le.pickle"
conf = 0.5

print("Loading face detector...")
prototxt = "model/deploy.prototxt"
model = "model/res10_300x300_ssd_iter_140000.caffemodel"
detector = cv2.dnn.readNetFromCaffe(prototxt, model)

print("Loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(embeddingModel)

recognizer = pickle.loads(open(recognizerFile, "rb").read())
le = pickle.loads(open(labelEncFile, "rb").read())

box = []
print("Starting video stream...")
cam = cv2.VideoCapture(0)
time.sleep(2.0)

while True:
    _, frame = cam.read()
    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]
    imageBlob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300),(104.0, 177.0, 123.0), swapRB=False, crop=False)

    detector.setInput(imageBlob)
    detections = detector.forward()

    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence > conf:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            if fW < 20 or fH < 20:
                continue

            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]
            text = "{}  : {:.2f}%".format(name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()

Loading face detector...
Loading face recognizer...
Starting video stream...


In [13]:
from collections import Iterable
import numpy as np
import imutils
import pickle
import time
import cv2
import csv


def flatten(lis):
    for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:
            yield item


embeddingFile = "output/embeddings.pickle"
embeddingModel = "openface_nn4.small2.v1.t7"
recognizerFile = "output/recognizer.pickle"
labelEncFile = "output/le.pickle"
conf = 0.5

print("[INFO] loading face detector...")
prototxt = "model/deploy.prototxt"
model = "model/res10_300x300_ssd_iter_140000.caffemodel"
detector = cv2.dnn.readNetFromCaffe(prototxt, model)

print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(embeddingModel)

recognizer = pickle.loads(open(recognizerFile, "rb").read())
le = pickle.loads(open(labelEncFile, "rb").read())

Roll_Number = ""
box = []
print("[INFO] starting video stream...")
cam = cv2.VideoCapture(0)
time.sleep(2.0)

while True:
    _, frame = cam.read()
    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    detector.setInput(imageBlob)
    detections = detector.forward()

    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence > conf:

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            if fW < 20 or fH < 20:
                continue

            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]
            with open('student.csv', 'r') as csvFile:
                reader = csv.reader(csvFile)
                for row in reader:
                    box = np.append(box, row)
                    name = str(name)
                    if name in row:
                        person = str(row)
                        print(name)
                listString = str(box)
                print(box)
                if name in listString:
                    singleList = list(flatten(box))
                    listlen = len(singleList)
                    Index = singleList.index(name)
                    name = singleList[Index]
                    Roll_Number = singleList[Index + 1]
                    print(Roll_Number)

            text = "{} : {} : {:.2f}%".format(name, Roll_Number, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                          (0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()

C:\Users\SHIVANI\AppData\Local\Temp\ipykernel_21268\3227147941.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


[INFO] loading face detector...
[INFO] loading face recognizer...
[INFO] starting video stream...
shivani2
['183.75568985939026' '205.3259700536728' '352.1944284439087'
 '446.8998223543167' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
34
shivani2
['181.08415603637695' '205.6806519627571' '352.4462342262268'
 '443.7407433986664' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
34
shivani
shivani
['178.4462571144104' '207.1017935872078' '352.8636932373047'
 '443.19255352020264' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani2
['179.22592163085938' '204.52109277248383' '349.73355531692505'
 '443.0826097726822' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
34
shivani2
['179.371976852417' '202.5261864066124' '350.24306774139404'
 '441.59143567085266' 'sanjay' '143050'

['160.6141448020935' '180.67511469125748' '326.08641386032104'
 '409.4085216522217' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['164.4561767578125' '182.12578743696213' '328.3362150192261'
 '410.01582741737366' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['165.8612608909607' '182.59097635746002' '329.7672986984253'
 '407.8509896993637' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['166.01276993751526' '182.26545006036758' '331.76050186157227'
 '409.67148542404175' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['166.18278622627258' '181.7578822374344' '331.5889835357666'
 '408.10719430446625' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34'

['153.33491563796997' '179.0543630719185' '319.6068048477173'
 '410.56251525878906' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['153.22201251983643' '178.55033576488495' '320.0445771217346'
 '410.17415821552277' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['154.55338954925537' '179.06663417816162' '321.5975046157837'
 '411.0654294490814' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['155.59526681900024' '179.75757122039795' '321.29184007644653'
 '411.2089276313782' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['155.81794381141663' '179.72911298274994' '321.6055154800415'
 '412.1764808893204' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34

['254.08864617347717' '250.2994269132614' '365.1587247848511'
 '394.24183666706085' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
34
shivani2
['260.2292060852051' '249.8841941356659' '368.3763027191162'
 '391.9325351715088' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
34
shivani
shivani
['260.0815773010254' '249.23617243766785' '369.87061500549316'
 '392.2347128391266' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani2
['261.65539026260376' '251.64758563041687' '370.608651638031'
 '392.10795164108276' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
34
shivani
shivani
['262.01387643814087' '250.021094083786' '371.1103320121765'
 '391.2087142467499' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivan

...

shivani
shivani
['393.3683753013611' '237.95029520988464' '545.4985499382019'
 '431.47210478782654' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['389.83200788497925' '237.67976760864258' '544.3620800971985'
 '431.28740787506104' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['389.6134614944458' '235.58289706707' '541.2989616394043'
 '429.5246332883835' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['382.7634930610657' '230.5066555738449' '531.3161730766296'
 '427.4588495492935' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['379.42328453063965' '230.5065482854843' '527.6403665542603'
 '427.81998217105865' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shiv

['306.07956647872925' '199.45118129253387' '442.58798360824585'
 '387.79723942279816' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['304.7445774078369' '199.0700125694275' '442.9452180862427'
 '387.6481354236603' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['306.4824342727661' '199.09750521183014' '442.6849365234375'
 '385.1470023393631' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['306.2717914581299' '199.50874149799347' '442.868971824646'
 '384.4178706407547' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['306.7463278770447' '199.11402761936188' '442.7648663520813'
 '386.5527480840683' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23


shivani
shivani
['312.7603054046631' '202.2836610674858' '449.2670774459839'
 '375.5497097969055' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['312.51204013824463' '201.9424170255661' '448.183536529541'
 '377.53280103206635' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['312.3516082763672' '201.70355290174484' '448.3041286468506'
 '378.0041992664337' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['312.9743814468384' '201.44512206315994' '447.4665641784668'
 '380.0717532634735' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['311.59011125564575' '201.08429789543152' '447.6309657096863'
 '377.0564138889313' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shiv

['313.51990699768066' '203.5033956170082' '449.12781715393066'
 '375.61901807785034' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['314.38164710998535' '204.22965735197067' '448.3856678009033'
 '376.740825176239' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['314.72121477127075' '203.9001613855362' '448.9320158958435'
 '375.56722462177277' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['315.32474756240845' '204.77354228496552' '448.0845093727112'
 '375.72021782398224' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['315.06872177124023' '204.05284613370895' '448.4942436218262'
 '376.6407787799835' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34'

['316.6908860206604' '205.85467368364334' '451.65427923202515'
 '376.1938691139221' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['315.7744288444519' '205.64808994531631' '452.1495223045349'
 '376.90755128860474' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['316.26609563827515' '205.5311992764473' '451.4957070350647'
 '377.18907594680786' 'sanjay' '143050' 'sanjay' '143050' 'sanjay'
 '143050' 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['315.3445601463318' '205.68179190158844' '451.04445219039917'
 '376.4696270227432' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
23
shivani
shivani
['316.8962359428406' '207.39128440618515' '450.6883978843689'
 '376.0302007198334' 'sanjay' '143050' 'sanjay' '143050' 'sanjay' '143050'
 'shivani' '23' 'shivani' '2' 'shivani2' '34']
